<a href="https://colab.research.google.com/github/mohitsshetty986/Movie-Recommendation-Systems/blob/master/Credits_Genres_Keywords_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Credits, Genres, and Keywords Based Recommender**
###**This recommender system is based on the following metadata: the 3 top actors, the director, related genres, and the movie plot keywords.**

In [7]:
import pandas as pd

In [8]:
from google.colab import files
uploaded=files.upload()

Saving credits.csv to credits.csv
Saving keywords.csv to keywords.csv
Saving movies_metadata.csv to movies_metadata.csv


####**We are going to use primarily two datasets 'Credits' and 'Keywords' to get the movie casts, genre and movie overview.**


In [61]:
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')
metadata=pd.read_csv('movies_metadata.csv',low_memory=False)

**We are going to load and merge them into main DataFrame 'metadata'.**

**We follow the below steps.**

In [62]:
credits.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [63]:
keywords.head(1)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."


In [64]:
metadata.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


**We need to convert the ids in 'keywords', 'credits' and 'metadata' dataframes to 'int' datatype inorder to merge the dataframes into 'metadata'**

In [67]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')


In [68]:
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

###**We can see the Cast, Crew and Keywords are added to the main dataframe**

In [70]:
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


###**The data is present in the form of "stringified" lists. We need to convert them into a way that is usable for us.**


In [69]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [71]:
import numpy as np

###**Get the director's name from the crew feature.**

In [72]:
def get_director(crew):
    for i in crew:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

##**Now we will write a function that will return the top 3 actors, top 3 keywords and top 3 genres of a movie.**

---


###**Note that a movie can have more than three actors and keywords but we are taking only the top three actors. If a movie has less than 3 actors and keywords then we are taking it all.**

###**Similarly a movie can belong to more than 3 genres but we are taking 3 atmost and if less then we are taking all.**

In [73]:
def get_list(x):                      # x :  cast, keywords, genres
    if isinstance(x, list):
        names = [i['name'] for i in x]

        if len(names) > 3:
            names = names[:3]
        return names

    return []

###**We are creating a director column and we get it's values using get_director funtion**

In [74]:
metadata['director'] = metadata['crew'].apply(get_director)

###**We are creating three more attributes for the metadata dataframe which are cast, keywords and genres.**

In [75]:
features = ['cast', 'keywords', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

##**We have the data we need for the movies recommendation : cast, director, keywords, genres.**

In [76]:
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


##**We will be now creating a string that contains all the above data(actors, director, genres and keywords) that we want to use in our vectorizer.**

**Note that before computing the similarity scores to get the recommendations we need to turn the names and keyword instances into lowercase and remove the spaces between words so that the vectorizer doesn't count the Tom of "Tom Hanks" and "Tom Hiddleston" as the same. So the actors with names having spaces will be represented as "TomHanks"**

In [77]:
def clean_data(x):         # x :  cast, keywords, genres, director
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [78]:
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [81]:
def required_columns(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + ' '.join(x['director']) + ' ' + ' '.join(x['genres'])

In [82]:
metadata['Required_data'] = metadata.apply(required_columns, axis=1)

In [83]:
metadata.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,director,Required_data
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[animation, comedy, family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy]",johnlasseter,jealousy toy boy tomhanks timallen donrickles ...


##**Now lets calculate the cosine similarities.** 

##**Here we are using CountVectorizer() instead of TF-IDF vectorizer which was used for movie plot based recommendation system. This is because we dont want to down-weight the actor or director's names if they appear in relatively more movies and it doesn't make any sense in this case.**

In [84]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['Required_data']) # fit_transform convert into numbers and store into a matrix

In [85]:
count_matrix.shape

(46628, 58204)

**In my case the dataset is huge and the google colab session gets crashed. So I'm taking a fraction of the metadata dataframe and working on it. I will have to go back and get the count_matrix for a fraction of metadata.**

In [89]:
fraction_metadata=metadata.head(10000)

In [90]:
count_matrix1 = count.fit_transform(fraction_metadata['Required_data'])

##**Compute the Cosine similarities**

In [93]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix1, count_matrix1)

**You're going to define a function that takes in a movie title as an input and outputs a list of the 10 most similar movies. First you need a reverse mapping of movie titles and DataFrame indices. In other words, you need to identify the index of a movie in your metadata DataFrame( for us it's fraction of metadata), given its title.**

In [92]:
indices = pd.Series(fraction_metadata.index, index=fraction_metadata['title'])

####**We get the final solution:** 
Follow these steps:


1.  Get the index of the movie given its title.

2.  Get the list of cosine similarity scores for that particular movie with all movies. Convert it into a list of tuples where the first element is its position, and the second is the similarity score.

3.  Sort the list based on the similarity scores.

4.  Get the top 10 elements of this list ignoring the first element as it refers to self (the movie most similar to a particular movie is the movie itself).

5.  Return the movie titles corresponding to the indices of the top elements.


In [95]:
def recommend_movies(title, cosine_sim=cosine_sim):
    
    idx = indices[title]

    
    sim_scores = list(enumerate(cosine_sim[idx]))

    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    
    sim_scores = sim_scores[1:11]

    
    movie_indices = [i[0] for i in sim_scores]

    
    return fraction_metadata['title'].iloc[movie_indices]

In [97]:
recommend_movies('The Godfather', cosine_sim)

1934           The Godfather: Part III
8001    The Night of the Following Day
7772                          Mitchell
5309                       The Gambler
5                                 Heat
426                      Carlito's Way
826                         Lotto Land
1084               Glengarry Glen Ross
1199            The Godfather: Part II
1430                     Donnie Brasco
Name: title, dtype: object